## Module 5 - Dans ce notebook Jupyter, l'objectif de productivité et les écarts de productivité sont calculés
* Etape 5a - Réglages
* Etape 5b - Calculer l'objectif de productivité 
* Etape 5c - Identifier les zones prospères
* Etape 5d - calculer les écarts de productivité 
**===========================================================================================================**
![title](img/Fig5.PNG)
**===========================================================================================================**

## Etape 5a - Réglages

## i) Importer les packages/librairies

In [1]:
import os
import glob

import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle                # A partir de mpl_toolkits.axes_grid1 importer make_axes_locatable
import matplotlib.lines as mlines                       # Aide à synchroniser la barre de couleur avec la taille du tracé
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable  # Aide à synchroniser la barre de couleur avec la taille du tracé
from matplotlib import pyplot as plt

# changer le répertoire à là ou les modules sont enregistrés
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## Etape 5b - Calculer l'objectif de productivité 

## i) Definir la fonction de traçage de l'objectif de productivité

In [2]:
def plotProductivityTargets (x, y, WP, title1,xlable1,ylable1,title2,xlable2,ylable2): 
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

    ax1.scatter(y,
                WP,
                marker='*',color="grey")

    ax1.scatter(np.nanpercentile(y, 95),
                np.nanpercentile(WP, 95),
                marker='*', color='black', s=100)

    ax1.axvline(np.nanpercentile(y, 95),
                color="#EE6666", linestyle="--")
    
    ax1.axhline(np.nanpercentile(WP, 95),
                color="#EE6666", linestyle="--")    
    
    # ax2 pour histogramme
    counts, bins, patches = ax2.hist(WP, bins=100, facecolor='skyblue', edgecolor='none', histtype="bar")

    # ajouter la légende 
    fakeLine = plt.Line2D([0,0], [0,1], color="#EE6666", linestyle='--')
    fakemark = mlines.Line2D([], [], color='black', marker='*', markersize=10)
    ax2.legend([fakeLine, fakemark], ["95 percentile", 'Productivity target'])

    # Coleurs des différents centiles de l'histogramme
    twentyfifth, ninetyfifth = np.nanpercentile(WP, [5, 95])
    for patch, leftside, rightside in zip(patches, bins[:-1], bins[1:]):
        if rightside < twentyfifth:
            patch.set_facecolor('#EE6666')
        elif leftside > ninetyfifth:
            patch.set_facecolor('green')

    # créer la legende
    handles = [Rectangle((0, 0), 1, 1, color=c, ec="k") for c in ['#EE6666', 'green']]
    labels = ["0-5 Percentile",">95 Percentile"]
    plt.legend(handles, labels)

    # Titre  
    ax1.set_title(title1, fontsize=14)
    ax1.set_xlabel(xlable1, fontsize=13)
    ax1.set_ylabel(ylable1, fontsize=13)

    ax2.set_title(title2, fontsize=14)
    ax2.set_xlabel(xlable2, fontsize=13)
    ax2.set_ylabel(ylable2, fontsize=13)

    return None 

## ii) Calculer l'objectif de biomasse et l'objectif de productivité en biomasse de l'eau

## * Importer les données d'entrée ( ${ET_a}$ saisonnière, biomasse et données raster de la PEb-WPb)

In [3]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

input_ETfolder    = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
input_ETfhs       = glob.glob(input_ETfolder + '\*.tif')

input_AGBMfolder  = os.path.join(dir_proj, dir_data, "4L2_Biomass") 
input_AGBMfhs     = glob.glob(input_AGBMfolder + '\*.tif')

input_WPbfolder   = os.path.join(dir_proj, dir_data, "4L2_WPb") 
input_WPbfhs      = glob.glob(input_WPbfolder + '\*.tif')

input_ETfhs , input_AGBMfhs, input_WPbfhs

## **  Calculer l'objectif de biomasse et l'objectif de PE biomasse

In [4]:
in_fh = input_ETfhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

Season        = []
BiomassTarget = []
WPbTarget     = []

for in_fhET, in_fhAGBM, in_fhWPb in zip(input_ETfhs, input_AGBMfhs, input_WPbfhs): 
    AETI   = gis.OpenAsArray(in_fhET, nan_values=True) 
    AGBM   = gis.OpenAsArray(in_fhAGBM,  nan_values=True) 
    WPb    = gis.OpenAsArray(in_fhWPb, nan_values=True)
    
    # Changer la forme des matrices
    AETI_1D    = np.reshape(AETI, AETI.shape[0]*AETI.shape[1])
    Biomass_1D = np.reshape(AGBM, AGBM.shape[0] * AGBM.shape[1])
    WPb_1D     = np.reshape(WPb, WPb.shape[0] * WPb.shape[1])
    
    # Objectifs de productivité
    Target_biomass = round(np.nanpercentile(Biomass_1D, 95), 0)  # Rendement au 95 centile
    TargetWP       = round(np.nanpercentile(WPb_1D, 95), 1)         # PE au 95 centile
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhET).split('.')[0].replace('AETI', '').replace('_', ' ')
    
    Season.append(date)
    BiomassTarget.append(Target_biomass)
    WPbTarget.append(TargetWP)
    
    # Digramme de dispersion du rendement vs PE et histogramme de la PE pour 2017
    year = int(date.split (' ')[1].split('-')[0])
    if year == 2017:
        title1  = 'Biomass vs WPb'+date
        xlable1 = 'Biomass [ton/ha]'
        ylable1 = 'WPb [kg/m3]'
        
        title2  = 'WPb'+date
        xlable2 = 'WPb [ton/ha]'
        ylable2 = 'Number of pixels'
        
        title1,xlable1,ylable1
        plotProductivityTargets(AETI_1D,  Biomass_1D, WPb_1D, title1,xlable1,ylable1, title2,xlable2,ylable2)

# enrgistrer les données en trame de données - forme tabulaire
data_tuples = list(zip(Season, BiomassTarget, WPbTarget))
df_targets = pd.DataFrame(data_tuples, columns=['Season', 'Target_biomass', 'Target_WPb'])
print(df_targets);

## iii) Calculer l'obejctif de rendement cultural et l'objectif de la productivité culturale de l'eau

## * Importer les données d'entrée (${ET_a}$ Saisonnière, rendement cultural et les données raster PEc-WPy)

In [5]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

input_ETfolder    = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
input_ETfhs       = glob.glob(input_ETfolder + '\*.tif')

input_Yieldfolder  = os.path.join(dir_proj, dir_data, "4L2_Yield") 
input_Yieldfhs     = glob.glob(input_Yieldfolder + '\*.tif')

input_WPyfolder   = os.path.join(dir_proj, dir_data, "4L2_WPy") 
input_WPyfhs      = glob.glob(input_WPyfolder + '\*.tif')

input_ETfhs , input_Yieldfhs, input_WPyfhs

## ** Calculer l'obejctif de rendement cultural et l'objectif de la PE de la culture

In [6]:
in_fh = input_ETfhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

Season        = []
YieldTarget   = []
WPyTarget     = []

year = 2009
for in_fhET, in_fhYield, in_fhWPy in zip(input_ETfhs, input_Yieldfhs, input_WPyfhs): 
    AETI   = gis.OpenAsArray(in_fhET, nan_values=True) 
    Yield  = gis.OpenAsArray(in_fhYield,  nan_values=True) 
    WPy    = gis.OpenAsArray(in_fhWPy, nan_values=True)
    
    # Changer la forme des matrices
    AETI_1D    = np.reshape(AETI, AETI.shape[0]*AETI.shape[1])
    Yield_1D   = np.reshape(Yield, Yield.shape[0] * Yield.shape[1])
    WPy_1D     = np.reshape(WPy, WPy.shape[0] * WPy.shape[1])
    
    # Objectifs de productivité
    Target_yield   = round(np.nanpercentile(Yield_1D, 95), 0)  # Yield at 95 percentile
    TargetWP       = round(np.nanpercentile(WPy_1D, 95), 1)         # WP at 95 percentile
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhET).split('.')[0].replace('AETI', '').replace('_', ' ')
    
    Season.append(date)
    YieldTarget.append(Target_yield)
    WPyTarget.append(TargetWP )
    
    # Digramme de dispersion du rendement vs PE et histogramme de la PE pour 2017
    year = int(date.split (' ')[1].split('-')[0])
    if year == 2017:
        title1  = 'Crop yield vs WPy'+date
        xlable1 = 'Crop yield[ton/ha]'
        ylable1 = 'WPy [kg/m3]'
        
        title2  = 'WPy'+date
        xlable2 = 'WPy [ton/ha]'
        ylable2 = 'Number of pixels'
        
        title1,xlable1,ylable1
        plotProductivityTargets(AETI_1D,  Yield_1D, WPy_1D, title1,xlable1,ylable1, title2,xlable2,ylable2)
        
    year += 1

# enrgistrer les données en trame de données - forme tabulaire
data_tuples = list(zip(Season, YieldTarget, WPyTarget))
df_targets = pd.DataFrame(data_tuples, columns=['Season', 'Target_yield', 'Target_WPy'])
print(df_targets);

## Etape 5c - Identifier les zones prospères 
* Les zones prospères sont identifiées sur la base de l'objectif de biomasse et l'objectif de la productivité en de l'eau (PEb

## i) Importer les données d'entrée (${ET_a}$ saisonnière, biomasse et données raster de la PEb-WPb)

In [7]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

input_ETfolder    = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
input_ETfhs       = glob.glob(input_ETfolder + '\*.tif')

input_AGBMfolder  = os.path.join(dir_proj, dir_data, "4L2_Biomass") 
input_AGBMfhs     = glob.glob(input_AGBMfolder + '\*.tif')

input_WPbfolder   = os.path.join(dir_proj, dir_data, "4L2_WPb") 
input_WPbfhs      = glob.glob(input_WPbfolder + '\*.tif')

input_ETfhs , input_AGBMfhs, input_WPbfhs

## ii) Cartographier les zones prospères de l'espace d'étude

In [8]:
in_fh = input_ETfhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for in_fhET, in_fhAGBM, in_fhWPb in zip(input_ETfhs, input_AGBMfhs, input_WPbfhs): 
    AETI   = gis.OpenAsArray(in_fhET, nan_values=True) 
    AGBM   = gis.OpenAsArray(in_fhAGBM,  nan_values=True) 
    WPb    = gis.OpenAsArray(in_fhWPb, nan_values=True)
    
    # Changer la forme des matrices
    AETI_1D    = np.reshape(AETI, AETI.shape[0]*AETI.shape[1])
    Biomass_1D = np.reshape(AGBM, AGBM.shape[0] * AGBM.shape[1])
    WPb_1D     = np.reshape(WPb, WPb.shape[0] * WPb.shape[1])
    
    # enrgistrer les données en trame de données - forme tabulaire
    data_tuples = list(zip(AETI_1D,Biomass_1D,WPb_1D))
    df          = pd.DataFrame(data_tuples, columns=['AETI','Biomass','WP'])
    
    # Seuils pour les objectifs
    TargetBiomass = round(np.nanpercentile(Biomass_1D, 95), 0)
    TargetWP      = round(np.nanpercentile(WPb_1D , 95), 1)
    maxBiomass    = round(df["Biomass"].max(),1)
    maxWPb        = round(df["WP"].max(),1)
    
# Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhET).split('.')[0].replace('AETI', '').replace('_', ' ')
    
    ## Tracer
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16, 8))
    
    # axe1, tracer la carte raster, mettre en évidence les Rendements supérieurs ou egaux à l'objectif
    cmap1   = colors.ListedColormap(['lightgrey', 'green'])
    bounds1 = [0, TargetBiomass, maxBiomass]
    norm1   = colors.BoundaryNorm(bounds1, cmap1.N)
    
    img1     = ax1.imshow(AGBM, cmap=cmap1, norm=norm1, extent=spatial_extent)
    divider1 = make_axes_locatable(ax1)
    cax1     = divider1.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img1, spacing='proportional', ax=ax1, cax=cax1)
    ax1.set_title ('Biomass(B) >= 95 percentile ' + date, fontsize=10)
    
    ### ajouter les titres des axes
    ax1.set_xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    ax1.set_ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    
    # axe2, tracer la carte raster, mettre en évidence les Rendements supérieurs ou egaux à l'objectif 
    cmap2   = colors.ListedColormap(['lightgrey', 'red'])
    bounds2 = [0, TargetWP, maxWPb]
    norm2   = colors.BoundaryNorm(bounds2, cmap2.N)
 
    img2     = ax2.imshow(WPb, cmap=cmap2, norm=norm2, extent=spatial_extent)
    divider2 = make_axes_locatable(ax2)
    cax2     = divider2.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img2, spacing='proportional', ax=ax2, cax=cax2)
    ax2.set_title ('B WP >= 95 percentile ' + date, fontsize=10)  
    
    # ax3, tracer la carte raster, mettre en évidence à la fois les rendements et PE supérieures ou égaux aux l'objectifs
    Biomass_targeSpots      = np.where(((AGBM >= TargetBiomass ) & (WPb >= TargetWP)), AGBM, np.nan)
    Biomass_WPb_targetSpots = np.where((AGBM  < TargetBiomass), 1, Biomass_targeSpots)

    cmap3   = colors.ListedColormap(['lightgrey', 'blue'])
    bounds3 = [0, TargetWP, maxWPb]
    norm3   = colors.BoundaryNorm(bounds3, cmap3.N)
 
    img3 = ax3.imshow(Biomass_WPb_targetSpots, cmap=cmap3, norm=norm3, extent=spatial_extent)
    # barre de couleurs de figure (img3, spacing='proportionnel', ax=ax3)
    ax3.set_title ('Biomass and WPb >= 95 percentile ' + date, fontsize=10)  
    
    plt.subplots_adjust(wspace=0.4)
    
    plt.show()
    ;

## Step 5d - Calculer les écarts de productivité
* i) Ecart de biomasse et écarts de la PE biomasse
* ii)Ecarts de rendement de la culture et écarts de la PE de la culture

## i) Ecart de biomasse et écarts de la productivité en biomasse de l'eau

## * Importer les données d'entrée (biomasse et données raster de la PEb-WPb)

In [9]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

input_AGBMfolder  = os.path.join(dir_proj, dir_data, "4L2_Biomass") 
input_AGBMfhs     = glob.glob(input_AGBMfolder + '\*.tif')

input_WPbfolder   = os.path.join(dir_proj, dir_data, "4L2_WPb") 
input_WPbfhs      = glob.glob(input_WPbfolder + '\*.tif')

input_AGBMfhs, input_WPbfhs

## * Dossier de sortie : Créer un ou connecter l'existant

In [10]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

output_folder1 = os.path.join(dir_proj, dir_data, "5L2_BiomassGaps") 
output_folder2 = os.path.join(dir_proj, dir_data, "5L2_WPbgaps") 

# si le repertoire et le dossier n'existent pas, en créer
if not os.path.exists(output_folder1):
    os.makedirs(output_folder1) 
if not os.path.exists(output_folder2):
    os.makedirs(output_folder2)

output_folder1, output_folder2

## ** Ecart de biomasse

In [11]:
in_fh = input_AGBMfhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for in_fhAGBM in input_AGBMfhs: 
    AGBM = gis.OpenAsArray(in_fhAGBM, nan_values=True) 
    
    # objectifs de productivité et écarts de produtivité
    Biomass_1D = np.reshape(AGBM, AGBM.shape[0] * AGBM.shape[1])  # Changer la forme des matrices
    TargetBiomass = round(np.nanpercentile(Biomass_1D, 95), 1)
    Biomassgap = np.where( AGBM < TargetBiomass, TargetBiomass - AGBM, np.nan)
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhAGBM).split('.')[0].replace('AGBM', '').replace('_', ' ')
    
    # Enregistrer l'écart de rendement
    output_folder = output_folder1
    basename      = os.path.basename(in_fhAGBM).replace('AGBM','BiomassGaps')
    output_fn     = os.path.join(output_folder, basename)
    gis.CreateGeoTiff(output_fn, Biomassgap, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Tracer la carte du raster
    seasonal = Biomassgap
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label ='Biomass gaps [ton/ha/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Biomass gaps [ton/ha/season] ' + date, fontsize=16)
    plt.show () 
    ;

## *** Ecarts de la PE de la Biomasse (WPb gaps)

In [12]:
in_fh = input_WPbfhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for in_fhWPb in input_WPbfhs: 
    WPb = gis.OpenAsArray(in_fhWPb, nan_values=True) 
    
    # objectifs de productivité et écarts de produtivité
    WPb_1D     = np.reshape(WPb, WPb.shape[0] * WPb.shape[1])  # Changer la forme des matrices
    TargetWPb  = round(np.nanpercentile(WPb_1D, 95), 2)
    WPbgap     = np.where( WPb < TargetWPb, TargetWPb - WPb, np.nan)
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhWPb).split('.')[0].replace('WPb', '').replace('_', ' ')
    
    # Enregistrer l'écart de rendement
    output_folder = output_folder2
    basename      = os.path.basename(in_fhWPb).replace('WPb','WPbGaps')
    output_fn     = os.path.join(output_folder, basename)
    gis.CreateGeoTiff(output_fn,  WPbgap, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Tracer la carte du raster
    seasonal =  WPbgap
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label = 'Biomass WP gaps [m3/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Biomass WP gaps [m3/season] ' + date, fontsize= 16)
    plt.show () 
    ;

## ii) Ecarts de rendement de la culture et écarts de productivité en eau de la culture

## * Importer les données d'entrée (biomasse et données raster de la PEb-WPb)

In [13]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

input_AGBMfolder  = os.path.join(dir_proj, dir_data, "4L2_Yield") 
input_Yieldfhs     = glob.glob(input_AGBMfolder + '\*.tif')

input_WPbfolder   = os.path.join(dir_proj, dir_data, "4L2_WPy") 
input_WPyfhs      = glob.glob(input_WPbfolder + '\*.tif')

input_Yieldfhs, input_WPbfhs

## ** Dossier de sortie : Créer un ou connecter l'existant

In [14]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

output_folder1 = os.path.join(dir_proj, dir_data, "5L2_YieldGaps") 
output_folder2 = os.path.join(dir_proj, dir_data, "5L2_WPygaps") 

# si le repertoire et le dossier n'existent pas, en créer
if not os.path.exists(output_folder1):
    os.makedirs(output_folder1) 
if not os.path.exists(output_folder2):
    os.makedirs(output_folder2)

output_folder1, output_folder2

## ** Ecarts de rendement de la culture

In [15]:
in_fh = input_Yieldfhs [0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for in_fhAYield in input_Yieldfhs : 
    Yield = gis.OpenAsArray(in_fhAYield, nan_values=True) 
    
    # objectifs de productivité et écarts de produtivité
    Yield_1D    = np.reshape(Yield, Yield.shape[0] * Yield.shape[1])  # Changer la forme des matrices
    TargetYield = round(np.nanpercentile(Yield_1D, 95), 1)
    Yieldgap    = np.where( Yield < TargetYield, TargetYield - Yield, np.nan)
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhAYield).split('.')[0].replace('Yield', '').replace('_', ' ')
    
    # Enregistrer l'écart de rendement
    output_folder = output_folder1
    basename      = os.path.basename(in_fhAYield).replace('Yield','YieldGaps')
    output_fn     = os.path.join(output_folder, basename)
    gis.CreateGeoTiff(output_fn, Yieldgap, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Tracer la carte du raster
    seasonal = Yieldgap
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label ='Crop yield gaps [ton/ha/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Crop yield gaps [ton/ha/season] ' + date, fontsize=16)
    plt.show () 
    ;

## *** Ecarts de la PE de la culture

In [16]:
in_fh = input_WPyfhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for in_fhWPy in input_WPyfhs: 
    WPy = gis.OpenAsArray(in_fhWPy, nan_values=True) 
    
    # objectifs de productivité et écarts de produtivité
    WPy_1D     = np.reshape(WPy, WPy.shape[0] * WPy.shape[1])  # Changer la forme des matrices
    TargetWPy  = round(np.nanpercentile(WPy_1D, 95), 2)
    WPygap     = np.where( WPy < TargetWPy, TargetWPy - WPy, np.nan)
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhWPy).split('.')[0].replace('WPy', '').replace('_', ' ')
    
    # Enregistrer l'écart de rendement
    output_folder = output_folder2
    basename      = os.path.basename(in_fhWPy).replace('WPy','WPyGaps')
    output_fn     = os.path.join(output_folder, basename)
    gis.CreateGeoTiff(output_fn,  WPbgap, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Tracer la carte du raster
    seasonal =  WPygap
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label = 'Crop WP gaps [m3/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Crop WP gaps [m3/season] ' + date, fontsize= 16)
    plt.show () 
    ;